In [3]:
import sqlite3
from pymongo import MongoClient
import os

In [ ]:
def sql_example():
    # Check database and remove so I can start clean
    if os.path.exists('test_database.db'):
        os.remove('test_database.db')

    # Connect to SQLite database
    db = sqlite3.connect('test_database.db')
    cursor = db.cursor()

    # Create tables: users and addresses
    cursor.execute('''CREATE TABLE users (
                        id INTEGER PRIMARY KEY,
                        name TEXT,
                        age INTEGER)''')

    cursor.execute('''CREATE TABLE addresses (
                        id INTEGER PRIMARY KEY,
                        user_id INTEGER,
                        address TEXT,
                        FOREIGN KEY (user_id) REFERENCES users (id))''')

    # Insert data into users table
    people = [
        ("Alice", 30),
        ("Bob", 25),
        ("Charlie", 35),
        ("David", 28),
        ("Eve", 40)
    ]
    cursor.executemany("INSERT INTO users (name, age) VALUES (?, ?)", people)

    # Insert data into addresses table
    cursor.execute("""INSERT INTO addresses (user_id, address) VALUES 
                   (1, "123 Apple St"),
                   (2, "456 Banana Ave"),
                   (3, "789 Cherry Blvd"),
                   (4, "555 Dragonfruit Way"),
                   (5, "757th Path")
                   """)

    # Commit changes
    db.commit()

    # Fetch data from users table (with ordering and limit)
    users = cursor.execute("SELECT * FROM users ORDER BY age").fetchall()
    print("\nSQL Users by age:")
    for user in users:
        print(user)

    # Fetch data from users with a condition (age > 30)
    users_above_30 = cursor.execute("SELECT * FROM users WHERE age > ?", (30,)).fetchall()
    print("\nUsers older than 30:")
    for user in users_above_30:
        print(user)

    # Fetch data using JOIN (users and addresses)
    cursor.execute('''SELECT users.name, users.age, addresses.address
                        FROM users
                        JOIN addresses ON users.id = addresses.user_id''')
    user_addresses = cursor.fetchall()
    print("\nUsers with addresses:")
    for user in user_addresses:
        print(user)

    # Update data: Update Bob's age to 26
    cursor.execute("UPDATE users SET age = ? WHERE name = ?", (26, "Bob"))
    db.commit()

    # Fetch updated user data
    updated_bob = cursor.execute("SELECT * FROM users WHERE name = ?", ("Bob",)).fetchall()
    print("\nUpdated Bob:")
    for user in updated_bob:
        print(user)

    # Delete data: Remove Alice
    cursor.execute("DELETE FROM users WHERE name = ?", ("Alice",))
    db.commit()

    # Fetch remaining users after deletion
    remaining_users = cursor.execute("SELECT * FROM users").fetchall()
    print("\nRemaining users after deletion:")
    for user in remaining_users:
        print(user)

    db.close()


In [22]:
def nosql_example():
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')  # Default connection URI for local MongoDB

    # Start with a clean database for testing
    dbnames = client.list_database_names()
    if 'test_database' in dbnames:
        client.drop_database('test_database')

    db = client['test_database']  # Create a database named 'test_database'
    
    # Create collections: users and addresses
    users_collection = db['users']  # Create a collection named 'users'
    addresses_collection = db['addresses']  # Create a collection named 'addresses'

    # Insert data into users collection
    users_data = [
        {"name": "Alice", "age": 30, "wage":35.35},
        {"name": "Bob", "age": 25},
        {"name": "Charlie", "age": 35},
        {"name": "David"},
        {"name": "Eve", "age": 40, "Favorite_color":{"Color":"Red"}}
    ]
    users_collection.insert_many(users_data)
    
    # Insert data into addresses collection (with references to users)
    addresses_data = [
        {"user_id": users_collection.find_one({"name": "Alice"})["_id"], "address": "123 Apple St"},
        {"user_id": users_collection.find_one({"name": "Bob"})["_id"], "address": "456 Banana Ave"},
        {"user_id": users_collection.find_one({"name": "Charlie"})["_id"], "address": "789 Cherry Blvd"},
        {"user_id": users_collection.find_one({"name": "David"})["_id"], "address": "555 Dragonfruit Way"},
        {"user_id": users_collection.find_one({"name": "Eve"})["_id"], "address": "757th Path"}
    ]
    addresses_collection.insert_many(addresses_data)

    # Fetch users ordered by age
    users = users_collection.find({"age": {'$exists':True}}).sort("age")
    print("\nMongoDB Users by age:")
    for user in users:
        print(user)

    # Fetch users older than 30
    users_above_30 = users_collection.find({"age": {"$gt": 30}})
    print("\nUsers older than 30:")
    for user in users_above_30:
        print(user)

    # Fetch users and their addresses by joining users and addresses
    user_addresses = []
    for user in users_collection.find({"age": {'$exists':True}}): # Check to see if they have an age in the collection
        address = addresses_collection.find_one({"user_id": user["_id"]}) # Connect the address to the user by their ID
        if address:
            user_addresses.append({"name": user["name"], "age": user["age"], "address": address["address"]})
    
    print("\nUsers with addresses:")
    for user in user_addresses:
        print(user)    

    # Update Bob's age to 26
    users_collection.update_one({"name": "Bob"}, {"$set": {"age": 26}})
    
    # Fetch updated Bob data
    updated_bob = users_collection.find({"name": "Bob"})
    print("\nUpdated Bob:")
    for user in updated_bob:
        print(user)

    # Delete Alice
    users_collection.delete_one({"name": "Alice"})
    
    # Fetch remaining users after deletion
    remaining_users = users_collection.find()
    print("\nRemaining users after deletion:")
    for user in remaining_users:
        print(user)

    # Close the MongoDB connection
    client.close()

In [17]:
print("SQL Example:")
sql_example()

SQL Example:

SQL Users by age:
(2, 'Bob', 25)
(4, 'David', 28)
(1, 'Alice', 30)
(3, 'Charlie', 35)
(5, 'Eve', 40)

Users older than 30:
(3, 'Charlie', 35)
(5, 'Eve', 40)

Users with addresses:
('Alice', 30, '123 Apple St')
('Bob', 25, '456 Banana Ave')
('Charlie', 35, '789 Cherry Blvd')
('David', 28, '555 Dragonfruit Way')
('Eve', 40, '757th Path')

Updated Bob:
(2, 'Bob', 26)

Remaining users after deletion:
(2, 'Bob', 26)
(3, 'Charlie', 35)
(4, 'David', 28)
(5, 'Eve', 40)


In [23]:
print("NoSQL Example:")
nosql_example()

NoSQL Example:

MongoDB Users by age:
{'_id': ObjectId('6753383017e2e85b20c0daa9'), 'name': 'Bob', 'age': 25}
{'_id': ObjectId('6753383017e2e85b20c0daa8'), 'name': 'Alice', 'age': 30, 'wage': 35.35}
{'_id': ObjectId('6753383017e2e85b20c0daaa'), 'name': 'Charlie', 'age': 35}
{'_id': ObjectId('6753383017e2e85b20c0daac'), 'name': 'Eve', 'age': 40, 'Favorite_color': {'Color': 'Red'}}

Users older than 30:
{'_id': ObjectId('6753383017e2e85b20c0daaa'), 'name': 'Charlie', 'age': 35}
{'_id': ObjectId('6753383017e2e85b20c0daac'), 'name': 'Eve', 'age': 40, 'Favorite_color': {'Color': 'Red'}}

Users with addresses:
{'name': 'Alice', 'age': 30, 'address': '123 Apple St'}
{'name': 'Bob', 'age': 25, 'address': '456 Banana Ave'}
{'name': 'Charlie', 'age': 35, 'address': '789 Cherry Blvd'}
{'name': 'Eve', 'age': 40, 'address': '757th Path'}

Updated Bob:
{'_id': ObjectId('6753383017e2e85b20c0daa9'), 'name': 'Bob', 'age': 26}

Remaining users after deletion:
{'_id': ObjectId('6753383017e2e85b20c0daa9')